In [1]:
import pandas as pd

In [2]:
file_path = '/content/data_process.csv'
df = pd.read_csv(file_path)

df.head()

,user_id,artist,track,playlist
0,9cc0cfd4d7d7885102480dd99e7a90d6,elvis costello,(the angels wanna wear my) red shoes,hard rock 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,elvis costello & the attractions,"(what's so funny 'bout) peace, love and unders...",hard rock 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,elvis costello & the attractions,accidents will happen,hard rock 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,elvis costello,alison,hard rock 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,lissie,all be okay,hard rock 2010


**1. Tiền xử lý dữ liệu**

Chuyển đổi user_id và track thành dạng số nguyên (dùng mã hóa ID).
Tạo cột rating bằng cách đếm số lần xuất hiện của bài hát trong danh sách phát của mỗi người dùng.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.ml.feature import StringIndexer

def preprocess_data(input_path, output_path):
    # Khởi tạo Spark Session
    spark = SparkSession.builder.appName("MusicPreprocessing").getOrCreate()

    # Đọc dữ liệu từ CSV
    df = spark.read.csv(input_path, header=True, inferSchema=True)

    # Mã hóa user_id và track thành dạng số
    user_indexer = StringIndexer(inputCol="user_id", outputCol="user_index")
    track_indexer = StringIndexer(inputCol="track", outputCol="track_index")

    df = user_indexer.fit(df).transform(df)
    df = track_indexer.fit(df).transform(df)

    # Tạo cột rating (số lần xuất hiện của bài hát trong danh sách phát)
    ratings_df = df.groupBy("user_index", "track_index").agg(count("*").alias("rating"))

    # Lưu dữ liệu đã xử lý ra file Parquet
    ratings_df.write.parquet(output_path, mode="overwrite")
    print(f"Processed data saved to {output_path}")

# Gọi hàm tiền xử lý
if __name__ == "__main__":
    preprocess_data("data_process.csv", "processed_data.parquet")


Processed data saved to processed_data.parquet


**2. Huấn luyện mô hình gợi ý**

Sử dụng thuật toán ALS (Alternating Least Squares) trong Spark MLlib.
Dự đoán danh sách bài hát được cá nhân hóa cho mỗi người dùng.

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

def train_model(input_path, model_path):
    # Khởi tạo Spark Session
    spark = SparkSession.builder.appName("MusicRecommendation").getOrCreate()

    # Đọc dữ liệu đã tiền xử lý
    ratings_df = spark.read.parquet(input_path)

    # Huấn luyện mô hình ALS
    als = ALS(
        userCol="user_index",
        itemCol="track_index",
        ratingCol="rating",
        nonnegative=True,
        implicitPrefs=True,
        rank=10,
        maxIter=10,
        regParam=0.1
    )

    model = als.fit(ratings_df)

    # Lưu mô hình
    model.write().overwrite().save(model_path)
    print(f"Model saved to {model_path}")

# Gọi hàm huấn luyện
if __name__ == "__main__":
    train_model("processed_data.parquet", "als_model")

Model saved to als_model
